In [0]:
%tensorflow_version 2.x

In [0]:
# Imports
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD
from imutils import paths
import tensorflow as tf
import numpy as np
import pathlib

In [3]:
print(tf.__version__)

2.0.0


In [0]:
# Get the flowers' dataset
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

In [5]:
!ls {flowers}

daisy  dandelion  LICENSE.txt  roses  sunflowers  tulips


In [0]:
CLASSES = ['daisy', 'dandelion', 'roses', 'sunflowers', 'tulips']

In [7]:
total_data = len(list(paths.list_images(flowers)))
total_data

3670

In [8]:
dataset = tf.data.Dataset.list_files(str(pathlib.Path(flowers)/'*/*'), seed=666)
for filename in dataset.take(5):
    print(filename.numpy())

b'/root/.keras/datasets/flower_photos/tulips/14087425312_2b5846b570_n.jpg'
b'/root/.keras/datasets/flower_photos/sunflowers/3846717708_ea11383ed8.jpg'
b'/root/.keras/datasets/flower_photos/sunflowers/244074259_47ce6d3ef9.jpg'
b'/root/.keras/datasets/flower_photos/dandelion/19812060274_c432f603db.jpg'
b'/root/.keras/datasets/flower_photos/sunflowers/3062794421_295f8c2c4e.jpg'


In [0]:
def parse_image(filename):
    parts = tf.strings.split(filename, '/')
    label = parts[-2]

    image = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(image)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, [128, 128])
    return (image, label)

In [0]:
dataset = dataset.map(parse_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [0]:
def _bytestring_feature(list_of_bytestrings):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=list_of_bytestrings))

def _int_feature(list_of_ints): # int64
    return tf.train.Feature(int64_list=tf.train.Int64List(value=list_of_ints))

In [0]:
def to_tfrecord(img_bytes, label):  
    class_num = np.argmax(np.array(CLASSES)==label) 
    feature = {
      "image": _bytestring_feature([img_bytes]), 
      "class": _int_feature([class_num]),             
    }
    return tf.train.Example(features=tf.train.Features(feature=feature))

In [0]:
def recompress_image(image, label):
    image = tf.cast(image, tf.uint8)
    image = tf.image.encode_jpeg(image, optimize_size=True, chroma_downsampling=False)
    return image, label

In [0]:
dataset = dataset.map(recompress_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.batch(32) 

In [15]:
for shard, (image, label) in enumerate(dataset):
    shard_size = image.numpy().shape[0]
    filename = "flowers-" + "{:02d}-{}.tfrec".format(shard, shard_size)
  
    with tf.io.TFRecordWriter(filename) as out_file:
        for i in range(shard_size):
            example = to_tfrecord(image.numpy()[i],label.numpy()[i])
            out_file.write(example.SerializeToString())
        print("Wrote file {} containing {} records".format(filename, shard_size))

Wrote file flowers-00-32.tfrec containing 32 records
Wrote file flowers-01-32.tfrec containing 32 records
Wrote file flowers-02-32.tfrec containing 32 records
Wrote file flowers-03-32.tfrec containing 32 records
Wrote file flowers-04-32.tfrec containing 32 records
Wrote file flowers-05-32.tfrec containing 32 records
Wrote file flowers-06-32.tfrec containing 32 records
Wrote file flowers-07-32.tfrec containing 32 records
Wrote file flowers-08-32.tfrec containing 32 records
Wrote file flowers-09-32.tfrec containing 32 records
Wrote file flowers-10-32.tfrec containing 32 records
Wrote file flowers-11-32.tfrec containing 32 records
Wrote file flowers-12-32.tfrec containing 32 records
Wrote file flowers-13-32.tfrec containing 32 records
Wrote file flowers-14-32.tfrec containing 32 records
Wrote file flowers-15-32.tfrec containing 32 records
Wrote file flowers-16-32.tfrec containing 32 records
Wrote file flowers-17-32.tfrec containing 32 records
Wrote file flowers-18-32.tfrec containing 32 r

In [0]:
def read_tfrecord(example):
    features = {
        "image": tf.io.FixedLenFeature([], tf.string), 
        "class": tf.io.FixedLenFeature([], tf.int64)
    }
    
    example = tf.io.parse_single_example(example, features)
    image = tf.image.decode_jpeg(example['image'], channels=3)
    image = tf.cast(image, tf.float32) / 255.0  
    image = tf.reshape(image, [128, 128, 3]) 
    class_label = tf.cast(example['class'], tf.int32)
    
    return (image, class_label)

In [0]:
def load_dataset(filenames):
    dataset = tf.data.Dataset.from_tensor_slices(filenames)
    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=16) 
    dataset = dataset.map(read_tfrecord, num_parallel_calls=tf.data.experimental.AUTOTUNE)
    
    return dataset

In [0]:
def batch_dataset(filenames, batch_size, train):
    dataset = load_dataset(filenames)
    
    if train:
        dataset = dataset.shuffle(buffer_size=1000).repeat()
    else:
        dataset = dataset.repeat()
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE) 
    return dataset

In [0]:
tfrecord_pattern = "*.tfrec"

In [20]:
filenames = tf.io.gfile.glob(tfrecord_pattern)
filenames[:10]

['./flowers-95-32.tfrec',
 './flowers-17-32.tfrec',
 './flowers-05-32.tfrec',
 './flowers-90-32.tfrec',
 './flowers-76-32.tfrec',
 './flowers-37-32.tfrec',
 './flowers-52-32.tfrec',
 './flowers-80-32.tfrec',
 './flowers-22-32.tfrec',
 './flowers-74-32.tfrec']

In [0]:
split = len(filenames) - int(len(filenames) * 0.1)
train_filenames = filenames[:split]
valid_filenames = filenames[split:]

In [0]:
BATCH_SIZE = 64

steps_per_epoch = (3670 - int(3670 * 0.1))//BATCH_SIZE
validation_steps = int(3670 * 0.1)//BATCH_SIZE

In [0]:
training_dataset = batch_dataset(train_filenames, BATCH_SIZE, True)
validation_dataset = batch_dataset(train_filenames, BATCH_SIZE, False)

In [0]:
def get_training_model():
    baseModel = VGG16(weights="imagenet", include_top=False,
        input_tensor=Input(shape=(128, 128, 3)))

    headModel = baseModel.output
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(512, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(5, activation="softmax")(headModel)

    model = Model(inputs=baseModel.input, outputs=headModel)

    for layer in baseModel.layers:
        layer.trainable = False

    opt = SGD(lr=1e-4, momentum=0.9)
    model.compile(loss="sparse_categorical_crossentropy", optimizer=opt,
        metrics=["accuracy"])
    return model

In [27]:
model = get_training_model()
model.fit(training_dataset, 
         steps_per_epoch=steps_per_epoch,
         validation_data=validation_dataset,
         validation_steps=validation_steps,
         epochs=5)

Train for 51 steps, validate for 5.0 steps
Epoch 1/5
51/51 [==============================] - 14s 282ms/step - loss: 0.1025 - accuracy: 0.9715 - val_loss: 4.9793e-04 - val_accuracy: 1.0000
Epoch 2/5
51/51 [==============================] - 10s 196ms/step - loss: 0.0013 - accuracy: 1.0000 - val_loss: 4.1703e-04 - val_accuracy: 1.0000
Epoch 3/5
51/51 [==============================] - 10s 194ms/step - loss: 0.0011 - accuracy: 1.0000 - val_loss: 3.7341e-04 - val_accuracy: 1.0000
Epoch 4/5
51/51 [==============================] - 10s 196ms/step - loss: 9.8622e-04 - accuracy: 1.0000 - val_loss: 3.3849e-04 - val_accuracy: 1.0000
Epoch 5/5
51/51 [==============================] - 10s 195ms/step - loss: 9.4331e-04 - accuracy: 1.0000 - val_loss: 3.0822e-04 - val_accuracy: 1.0000
